In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
!pip install plotly
import plotly.express as px
import plotly.graph_objects as go

In [1]:
import pandas as pd
# pd.options.plotting.backend = "plotly"

# 제주도 연령별 인구현황(남,여 구분) 데이터 불러오기 

2개의 csv 파일 각각 불러오기
* 201512_202012_연령별인구현황_연간.csv
* 200912_201412_연령별인구현황_연간.csv

In [5]:
df1 = pd.read_csv('/content/drive/MyDrive/울산대학교 AI 고급 - LSTM/01. DNN 복습 실습/00. Pandas 활용 능력 평가/제주도_삼다도/data/200912_201412_연령별인구현황_연간.csv', encoding='cp949', thousands=',')
df2 = pd.read_csv('/content/drive/MyDrive/울산대학교 AI 고급 - LSTM/01. DNN 복습 실습/00. Pandas 활용 능력 평가/제주도_삼다도/data/201512_202012_연령별인구현황_연간.csv', encoding='cp949', thousands=',')

In [10]:
data = pd.merge(df1, df2)
jeju = data.copy()
jeju.head()

,행정구역,2009년_계_총인구수,2009년_계_연령구간인구수,2009년_계_0~9세,2009년_계_10~19세,2009년_계_20~29세,2009년_계_30~39세,2009년_계_40~49세,2009년_계_50~59세,2009년_계_60~69세,2009년_계_70~79세,2009년_계_80~89세,2009년_계_90~99세,2009년_계_100세 이상,2009년_남_총인구수,2009년_남_연령구간인구수,2009년_남_0~9세,2009년_남_10~19세,2009년_남_20~29세,2009년_남_30~39세,2009년_남_40~49세,2009년_남_50~59세,2009년_남_60~69세,2009년_남_70~79세,2009년_남_80~89세,2009년_남_90~99세,2009년_남_100세 이상,2009년_여_총인구수,2009년_여_연령구간인구수,2009년_여_0~9세,2009년_여_10~19세,2009년_여_20~29세,2009년_여_30~39세,2009년_여_40~49세,2009년_여_50~59세,2009년_여_60~69세,2009년_여_70~79세,2009년_여_80~89세,2009년_여_90~99세,2009년_여_100세 이상,...,2019년_여_100세 이상,2020년_계_총인구수,2020년_계_연령구간인구수,2020년_계_0~9세,2020년_계_10~19세,2020년_계_20~29세,2020년_계_30~39세,2020년_계_40~49세,2020년_계_50~59세,2020년_계_60~69세,2020년_계_70~79세,2020년_계_80~89세,2020년_계_90~99세,2020년_계_100세 이상,2020년_남_총인구수,2020년_남_연령구간인구수,2020년_남_0~9세,2020년_남_10~19세,2020년_남_20~29세,2020년_남_30~39세,2020년_남_40~49세,2020년_남_50~59세,2020년_남_60~69세,2020년_남_70~79세,2020년_남_80~89세,2020년_남_90~99세,2020년_남_100세 이상,2020년_여_총인구수,2020년_여_연령구간인구수,2020년_여_0~9세,2020년_여_10~19세,2020년_여_20~29세,2020년_여_30~39세,2020년_여_40~49세,2020년_여_50~59세,2020년_여_60~69세,2020년_여_70~79세,2020년_여_80~89세,2020년_여_90~99세,2020년_여_100세 이상
0,제주특별자치도 (5000000000),562663,562663,63916,83008,70590,89763,96372,67449,47146,31830,10533,1981,75,281319,281319,33434,43635,36642,46066,50381,34169,22530,12038,2186,233,5,281344,281344,30482,39373,33948,43697,45991,33280,24616,19792,8347,1748,70,...,236,674635,674635,60297,70661,82746,82555,112376,112401,79761,45591,23892,4059,296,338609,338609,30968,36968,43708,41788,57969,58346,39555,20694,7975,598,40,336026,336026,29329,33693,39038,40767,54407,54055,40206,24897,15917,3461,256
1,제주특별자치도 제주시 (5011000000),410378,410378,48600,62660,52297,67708,71030,47711,31282,20608,7087,1341,54,204689,204689,25459,33090,27061,33971,36612,24051,15011,7830,1452,148,4,205689,205689,23141,29570,25236,33737,34418,23660,16271,12778,5635,1193,50,...,146,492466,492466,45724,53274,63596,62686,82912,80753,54878,30090,15577,2778,198,246588,246588,23476,27869,33453,31771,42197,41449,27149,13603,5194,400,27,245878,245878,22248,25405,30143,30915,40715,39304,27729,16487,10383,2378,171
2,제주특별자치도 서귀포시 (5013000000),152285,152285,15316,20348,18293,22055,25342,19738,15864,11222,3446,640,21,76630,76630,7975,10545,9581,12095,13769,10118,7519,4208,734,85,1,75655,75655,7341,9803,8712,9960,11573,9620,8345,7014,2712,555,20,...,90,182169,182169,14573,17387,19150,19869,29464,31648,24883,15501,8315,1281,98,92021,92021,7492,9099,10255,10017,15772,16897,12406,7091,2781,198,13,90148,90148,7081,8288,8895,9852,13692,14751,12477,8410,5534,1083,85


In [11]:
jeju.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 0 to 2
Columns: 469 entries, 행정구역 to 2020년_여_100세 이상
dtypes: int64(468), object(1)
memory usage: 11.0+ KB


# 데이터 가공

In [12]:
# 컬럼 -> 행으로 전환
jeju = jeju.set_index('행정구역').stack(0).reset_index()

In [13]:
# 행정구역 수정
jeju.행정구역 = jeju.행정구역.str.split('(').str[0].str.strip()

In [14]:
# 컬럼 분리 연도, 성별, 연령대
year = jeju.level_1.str.split('년_').str[0]
tmp = jeju.level_1.str.split('년_').str[1].str.split("_", expand=True)

In [15]:
# "level_1" 컬럼을 분리하여 "연도", "성별", "연령대" 컬럼 만들기
year = jeju.level_1.str.split('년_').str[0]
tmp = jeju.level_1.str.split('년_').str[1].str.split("_", expand=True)

# 결합
jeju = pd.concat([jeju['행정구역'], year, tmp, jeju[0]], axis=1)
jeju.columns = ['행정구역','연도','성별','연령','인구']

In [17]:
jeju.describe(include='all')

,행정구역,연도,성별,연령,인구
count,1296,1296,1296,1296,1296.000000
unique,3,12,3,12,NaN
top,제주특별자치도,2017,남,80~89세,NaN
freq,432,108,432,108,NaN
mean,NaN,NaN,NaN,NaN,45869.314815
std,NaN,NaN,NaN,NaN,87178.990960
min,NaN,NaN,NaN,NaN,1.000000
25%,NaN,NaN,NaN,NaN,7777.750000
50%,NaN,NaN,NaN,NaN,22203.500000
75%,NaN,NaN,NaN,NaN,43488.500000


In [18]:
jeju.head()

,행정구역,연도,성별,연령,인구
0,제주특별자치도,2009,계,총인구수,562663
1,제주특별자치도,2009,계,0~9세,63916
2,제주특별자치도,2009,계,10~19세,83008
3,제주특별자치도,2009,계,20~29세,70590
4,제주특별자치도,2009,계,30~39세,89763


# 데이터 시각화

* Plotly 공식문서: https://plotly.com/python/

## 2020년 제주시 남녀 성비 시각화

In [21]:
region = jeju['행정구역'] == '제주특별자치도'
year = jeju['연도'] == "2020"
gender = jeju['성별'] != '계'
age = jeju['연령'] == '총인구수'

fig = px.pie(jeju[region&year&gender&age], 
             values='인구', 
             names='성별',
             color_discrete_sequence=px.colors.sequential.RdBu,
             title="2020년 제주시 남녀 성비",
             height=800
            )
fig.show()

## 제주시 남녀 인구 차이 시각화

In [22]:
reg = '제주특별자치도'
region = jeju['행정구역'] == reg
gender = jeju['성별'] != '계'
age = jeju['연령'] == '총인구수'

gender_gap = jeju[region&gender&age].sort_values(['연도','성별'], ascending=(True, False))

tmp = pd.DataFrame(
    dict(연도=gender_gap.연도.unique(),
         남녀격차=gender_gap[::2]['인구'].values - gender_gap[1::2]['인구'].values,
        )
)

fig= px.bar(tmp, 
            x='연도', 
            y='남녀격차',
            title=f'{reg} 남녀 인구 격차 변화 (여성 인구 - 남성 인구)',
            color='남녀격차',
            height=600
           )
fig.show()

In [23]:
reg = '제주특별자치도 제주시'
region = jeju['행정구역'] == reg
gender = jeju['성별'] != '계'
age = jeju['연령'] == '총인구수'

gender_gap = jeju[region&gender&age].sort_values(['연도','성별'], ascending=(True, False))
jj_gap = pd.DataFrame(
    dict(연도=gender_gap.연도.unique(),
         남녀격차=gender_gap[::2]['인구'].values - gender_gap[1::2]['인구'].values,
        )
)

fig= px.bar(jj_gap, 
            x='연도', 
            y='남녀격차',
            title=f'{reg} 남녀 인구 격차 변화 (여성 인구 - 남성 인구)',
            color='남녀격차',
            height=600
           )
fig.show()

In [24]:
reg = '제주특별자치도 서귀포시'
region = jeju['행정구역'] == reg
gender = jeju['성별'] != '계'
age = jeju['연령'] == '총인구수'

gender_gap = jeju[region&gender&age].sort_values(['연도','성별'], ascending=(True, False))
sg_gap = pd.DataFrame(
    dict(연도=gender_gap.연도.unique(),
         남녀격차=gender_gap[::2]['인구'].values - gender_gap[1::2]['인구'].values,
        )
)

fig= px.bar(sg_gap, 
            x='연도', 
            y='남녀격차',
            title=f'{reg} 남녀 인구 격차 변화 (여성 인구 - 남성 인구)',
            color='남녀격차',
            height=600
           )
fig.show()

## graph_object

In [25]:
import plotly.graph_objects as go

In [26]:
data = [go.Bar(name='제주시', x=jj_gap.연도, y=jj_gap.남녀격차, marker_color='rgb(55, 83, 109)'),
        go.Bar(name='서귀포시', x=sg_gap.연도, y=sg_gap.남녀격차, marker_color='rgb(26, 118, 255)')]

fig = go.Figure(data)
fig.update_layout(
    title='제주시 - 서귀포시 남녀 인구격차(여-남) 변화',
    yaxis=dict(
        title='인구격차'),
    xaxis=dict(
        title='연도'),
    height=600)
fig.show()